In [3]:
with open('Assignment2_train.txt','r') as f:
    x = f.readlines()
x

["'march 8 1758', '1758-03-08'\n",
 "'17 february 1709', '1709-02-17'\n",
 "'13 may 1786', '1786-05-13'\n",
 "'17 june 1626', '1626-06-17'\n",
 "'july 25 1851', '1851-07-25'\n",
 "'1975 4 mar', '1975-03-04'\n",
 "'16 march 1926', '1926-03-16'\n",
 "'11 january 2043', '2043-01-11'\n",
 "'sat 14 apr 2007', '2007-04-14'\n",
 "'thu 1873 30 october', '1873-10-30'\n",
 "'october 14 1584', '1584-10-14'\n",
 "'may 4 1942', '1942-05-04'\n",
 "'13/09/1650', '1650-09-13'\n",
 "'24 july 1572', '1572-07-24'\n",
 "'september 24 1946', '1946-09-24'\n",
 "'jul 24 2030', '2030-07-24'\n",
 "'tuesday 22 february 1853', '1853-02-22'\n",
 "'november 24 1700', '1700-11-24'\n",
 "'2 december 1633', '1633-12-02'\n",
 "'tuesday 17 april 1798', '1798-04-17'\n",
 "'september 10 1607', '1607-09-10'\n",
 "'monday 1918 4 02', '1918-02-04'\n",
 "'aug 29 1827', '1827-08-29'\n",
 "'2014 13 mar', '2014-03-13'\n",
 "'mar 13 1560', '1560-03-13'\n",
 "'4 september 2046', '2046-09-04'\n",
 "'november 28 1576', '1576-11-28'

In [4]:
text = " ' Hello world ' '"
print(text)
stripped_chars = set(["'", " "])
stripped_chars_str = "' \"\\n"
text = text.strip(stripped_chars_str)
print(text)

 ' Hello world ' '
Hello world


In [5]:
stripped_chars = set(["'", " "])
stripped_chars

{' ', "'"}

In [6]:
x[0].split(",")[0]

"'march 8 1758'"

In [7]:
temp = x[0].split(",")[0]
print(temp.strip(stripped_chars_str))

march 8 1758


In [8]:
type(temp.strip(stripped_chars_str))

str

In [9]:
y = []
for data in x:
    data_hd = data.split(",")[0]
    data_md = data.split(",")[1]
    data_hd = data_hd.strip(stripped_chars_str)
    data_md = data_md.strip("\n")
    data_md = data_md.strip(stripped_chars_str)
    y.append([data_hd, data_md])
print(y[0:5])

[['march 8 1758', '1758-03-08'], ['17 february 1709', '1709-02-17'], ['13 may 1786', '1786-05-13'], ['17 june 1626', '1626-06-17'], ['july 25 1851', '1851-07-25']]


In [10]:
vocab = set()
for dates in y:
    for char in dates[0]:
        vocab.add(char)
    for char in dates[1]:
        vocab.add(char)
vocab.add("<sos>")
vocab.add("<eos>")
vocab.add("<pad>")
vocab.add("<unk>")
print(vocab)

{'<pad>', 'y', ' ', '/', '6', 'v', 'w', '<unk>', 'i', '2', 'f', 'g', 'n', 'l', '9', 'd', 's', 'r', 'o', '-', 'u', 'h', '5', '3', 'e', '8', 'b', 'j', '<eos>', 'a', '4', 'm', 'c', '1', '<sos>', 'p', '0', 't', '7'}


In [11]:
vocab_list = list(vocab)
print(vocab_list)

['<pad>', 'y', ' ', '/', '6', 'v', 'w', '<unk>', 'i', '2', 'f', 'g', 'n', 'l', '9', 'd', 's', 'r', 'o', '-', 'u', 'h', '5', '3', 'e', '8', 'b', 'j', '<eos>', 'a', '4', 'm', 'c', '1', '<sos>', 'p', '0', 't', '7']


In [12]:
vocab_list.sort()

In [13]:
print(vocab_list)

[' ', '-', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '<eos>', '<pad>', '<sos>', '<unk>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [14]:
len("Rajkamal is a good boy")

22

In [15]:
#Calculating max_seq_len possible of human dates
mx = -1
for dates in y:
    mx = max(mx, len(dates[0]))
mx

27

So, my transformer architecture's max_seq_len is going to be 32.
Decoder's output max_seq_len need not be equal to that of encoder's, in my case, as the output is always going to be YYYY-MM-DD so, I will be generating 11 characters in the output via decoder (one extra character for end token - can skip it, depends, check it later).

In [16]:
ctoi = {ch:i for i,ch in enumerate(vocab_list)}
print(ctoi)

{' ': 0, '-': 1, '/': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '<eos>': 13, '<pad>': 14, '<sos>': 15, '<unk>': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'l': 27, 'm': 28, 'n': 29, 'o': 30, 'p': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'y': 38}


In [17]:
itoc = {i:ch for i,ch in enumerate(vocab_list)}
print(itoc)
#will be needed while decoding

{0: ' ', 1: '-', 2: '/', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: '<eos>', 14: '<pad>', 15: '<sos>', 16: '<unk>', 17: 'a', 18: 'b', 19: 'c', 20: 'd', 21: 'e', 22: 'f', 23: 'g', 24: 'h', 25: 'i', 26: 'j', 27: 'l', 28: 'm', 29: 'n', 30: 'o', 31: 'p', 32: 'r', 33: 's', 34: 't', 35: 'u', 36: 'v', 37: 'w', 38: 'y'}


Standard word embedding dimensions I encountered during word2vec, glove and fasttext was around 300, whereas for all-mpnet-base-v2, it was 768, so, accordingly scaling for character embedding dimensions - I believe 100 dim is good starting point for now!<br>
So, my d_model = 100!

In [18]:
import torch
import torch.nn as nn

In [18]:
class Encoder(nn.Module):#nn.Module is needed to have the powers of nn i.e. BACKPROPAGATION, Computation graph, etc
    def __init__(self):#constructor
        super().__init__() #calling constructor of nn.Module to have all it's functionalities available to me
        self.embedding = nn.Embedding(39, 100, padding_idx=14) 
        #self.embedding[14] = torch.zeroes()
        with torch.no_grad():
            self.embedding.weight[14] = torch.zeros(100)

encoder = Encoder()
#print(encoder.embedding.shape)
print(encoder.embedding.weight.shape)
#print(encoder.embedding[14])
print(encoder.embedding.weight[14])
print(encoder.embedding.weight[1])
    
    

torch.Size([39, 100])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], grad_fn=<SelectBackward0>)
tensor([-1.0646,  1.6308,  1.5996, -1.9984,  0.2420, -0.6331,  0.1340, -1.5962,
        -0.5247, -1.1418,  0.0591,  1.5399,  0.3686, -0.6437,  0.0927, -0.5058,
         0.9930, -1.8410,  0.0176,  1.2356, -1.9174,  0.3533,  0.6007, -2.1403,
         1.8278, -0.5869, -1.9603, -1.8965, -0.4448, -0.4983, -0.5537,  0.4554,
         0.9840, -0.4013,  0.1496, -0.6410, -1.4493, -0.3307,  0.5997, -0.0778,
         0.3949,  0.5148,  0.1501, -0.4114, -0.0235,  0.0398, -0.7414,  1.7041,
        -1.6444,  1.1851,  0.7

In [83]:
temp = torch.randn(4,4)
rows = [1,2]
print(temp)
print(temp[rows])
print(temp[:][1])
print(temp.shape)
temp[:,1]

tensor([[ 1.0984,  1.3883, -0.7145,  0.6942],
        [-0.9190,  1.5489, -0.5259,  0.9685],
        [-2.3442,  0.5018, -0.8843,  0.9437],
        [ 0.1609, -0.0823,  0.7944,  1.3473]])
tensor([[-0.9190,  1.5489, -0.5259,  0.9685],
        [-2.3442,  0.5018, -0.8843,  0.9437]])
tensor([-0.9190,  1.5489, -0.5259,  0.9685])
torch.Size([4, 4])


tensor([ 1.3883,  1.5489,  0.5018, -0.0823])

In [96]:
MAX_SEQ_LEN = 32
EMBEDDING_DIM = 128 #100 changed, since, I want it to be divisible by 4
#EMBEDDING_DIM is the d_model we talked about while discussing architecture
PAD_IDX = ctoi["<pad>"]
VOCAB_SIZE = len(vocab_list)
ATTENTION_HEADS = 2

In [130]:
temp = torch.randint(0,10,(3,4))
temp *= 10
temp

import math
math.sqrt(100)

10.0

In [133]:
temp = torch.randn(2,3)
softmax = nn.Softmax(dim=1)#across row - sum = 1
softmax(temp)

tensor([[0.0323, 0.2477, 0.7201],
        [0.6454, 0.2040, 0.1507]])

In [153]:
import torch

# Two 1D tensors
temp = torch.randn(4)
temp2 = torch.randn(4)
print(temp)
print(temp2)

# Concatenate them along dimension 0
result = torch.cat((temp, temp2), dim=0)

print(result.shape)
print(result)


tensor([ 0.2437,  0.3785,  1.3521, -1.0081])
tensor([ 0.6855, -0.7513,  0.0704,  0.1691])
torch.Size([8])
tensor([ 0.2437,  0.3785,  1.3521, -1.0081,  0.6855, -0.7513,  0.0704,  0.1691])


In [223]:
#FOR PARALLELIZING computations in multi-head attention across n heads

temp = torch.randint(0,10,(2,2))
print(temp)
temp2 = torch.randint(0,10,(2,2))
print(temp2)
temp3 = torch.cat((temp, temp2), dim = 0)
print(temp3)
print(temp3.shape)
#2*2 matrix and another 2*2 matrix - concatenate them to get 2 * 2*2 matrix
temp4 = temp3.view(-1,2,2)
print(temp4)
print(temp4.shape)


temp5 = torch.randint(0,10, (2,2))
print(temp5)
temp6 = torch.randint(0,10,(2,2))
print(temp6)

temp7 = torch.cat((temp, temp2, temp5, temp6), dim = 0)
print(temp7)
print(temp7.shape)

temp8 = temp7.view(-1, 2, 2)
print(temp8)
print(temp8.shape)

temp9 = torch.randint(0,10, (2,2))
print(temp9)
temp10 = temp8 @ temp9
print("TEMP10: ",temp10)
print(temp10.shape)

temp9 = temp9.view(-1, 2, 2)
temp11 = temp8 @ temp9
print("TEMP11: ",temp11)
print(temp11.shape)

temp12 = temp9 @ temp8
print(temp12)
print(temp12.shape)




tensor([[7, 3],
        [2, 0]])
tensor([[5, 2],
        [8, 3]])
tensor([[7, 3],
        [2, 0],
        [5, 2],
        [8, 3]])
torch.Size([4, 2])
tensor([[[7, 3],
         [2, 0]],

        [[5, 2],
         [8, 3]]])
torch.Size([2, 2, 2])
tensor([[0, 9],
        [4, 0]])
tensor([[2, 5],
        [7, 5]])
tensor([[7, 3],
        [2, 0],
        [5, 2],
        [8, 3],
        [0, 9],
        [4, 0],
        [2, 5],
        [7, 5]])
torch.Size([8, 2])
tensor([[[7, 3],
         [2, 0]],

        [[5, 2],
         [8, 3]],

        [[0, 9],
         [4, 0]],

        [[2, 5],
         [7, 5]]])
torch.Size([4, 2, 2])
tensor([[3, 6],
        [2, 4]])
TEMP10:  tensor([[[27, 54],
         [ 6, 12]],

        [[19, 38],
         [30, 60]],

        [[18, 36],
         [12, 24]],

        [[16, 32],
         [31, 62]]])
torch.Size([4, 2, 2])
TEMP11:  tensor([[[27, 54],
         [ 6, 12]],

        [[19, 38],
         [30, 60]],

        [[18, 36],
         [12, 24]],

        [[16, 32],
    

In [147]:
import math
import torch
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, max_seq_len = MAX_SEQ_LEN, ip_embedding_dim = EMBEDDING_DIM, proj_embedding_dim = EMBEDDING_DIM):
        super().__init__()
        #self.W_q = torch.randn(max_seq_len, embedding_dim)
        #self.W_k = torch.randn(max_seq_len, embedding_dim)
        #self.W_v = torch.randn(max_seq_len, embedding_dim)
        self.W_q = torch.randn(ip_embedding_dim, proj_embedding_dim)
        self.W_k = torch.randn(ip_embedding_dim, proj_embedding_dim)
        self.W_v = torch.randn(ip_embedding_dim, proj_embedding_dim)

    def forward(self, token_embeddings):
        Q = token_embeddings @ self.W_q
        K = token_embeddings @ self.W_k
        V = token_embeddings @ self.W_v
        print(K.shape[1])
        Q_K = (Q @ K.T)/math.sqrt(K.shape[1])

        #My V matrix: is 32 * 100, so I will have to do across-row softmax, if it was 100 * 32, then, across-col softmax would have worked
        #NOTE: dim = 0 => row dim, dim = 1 => 
        softmax = nn.Softmax(dim=1)
        softmax_op = softmax(Q_K)

        print("SOFTMAX OP: ",softmax_op.shape)

        output = softmax_op @ V

        print(output.shape)

        return output

        #print(Q_K.shape)
        
        #print(Q.shape)
        #print(K.shape)
        #print(V.shape)
        #print(self.W_q.shape)
        #print(token_embeddings.shape)
        #return token_embeddings

class Multi_Head_Attention(nn.Module):
    def __init__(self, heads = ATTENTION_HEADS, ip_embedding_dim = EMBEDDING_DIM, max_seq_len = MAX_SEQ_LEN):
        super().__init__()
        self.heads = heads
        print("REDUCED DIMENSION: ", int(ip_embedding_dim/self.heads))
        self.attention_heads = [Attention(ip_embedding_dim = int(ip_embedding_dim/self.heads), proj_embedding_dim = int(ip_embedding_dim/self.heads)) for i in range(heads)]
        self.W_qs = [head.W_q for head in self.attention_heads]#learnable
        self.W_ks = [head.W_k for head in self.attention_heads]#learnable
        self.W_vs = [head.W_v for head in self.attention_heads]#learnable
        self.ip_embedding_dim = ip_embedding_dim
        self.proj_embedding_dim = int(ip_embedding_dim/heads)
        self.token_projection = nn.Linear(ip_embedding_dim, self.proj_embedding_dim)
        self.max_seq_len = max_seq_len
        self.linear = nn.Linear(ip_embedding_dim, ip_embedding_dim)
        #self.mask = torch.zeros(max_seq_len * max_seq_len)#NOT LEARNABLE
        #self.mask[:, -num_padded_tokens] = -float('inf')

    def forward(self, token_embeddings, num_padded_tokens):
        #I was thinking of computing contextually aware repr from each of the attention heads one-by-one and then concatenating them
        #This will work and it's implementation is also pretty striaghtforward, but this defeats the purpose of parallelization
        #which can be achieved across the attention heads
        W_qs_ = torch.cat(tuple(self.W_qs), dim = 0)
        W_ks_ = torch.cat(tuple(self.W_ks), dim = 0)
        W_vs_ = torch.cat(tuple(self.W_vs), dim = 0)
        print("EYLLO")
        print(W_qs_.shape)
        print(W_ks_.shape)
        print(W_vs_.shape)

        print("EYLLOSLKDJFLS")

        W_qs_ = W_qs_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim)
        W_ks_ = W_ks_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim)
        W_vs_ = W_vs_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim)

        print(W_qs_.shape)
        print(W_ks_.shape)
        print(W_vs_.shape)

        print(token_embeddings.shape)

        token_embeddings_proj = self.token_projection(token_embeddings)

        print(token_embeddings_proj.shape)

        print("W_qs_ shape: ",W_qs_.shape)
        print("token_embeddings_proj shape: ",token_embeddings_proj.shape)
        print("token_embeddings_proj transpose shape: ",token_embeddings_proj.T.shape)

        #Qs = W_qs_ @ token_embeddings_proj.T
        #print(Qs.shape)
        #Ks = W_ks_ @ token_embeddings_proj.T
        #print(Ks.shape)
        squeezed = token_embeddings_proj.T.unsqueeze(0)
        print("token_embeddings_proj.unsqueeze: ", squeezed.shape )
        Qs = torch.matmul(W_qs_, squeezed)
        print(Qs.shape)
        Ks = torch.matmul(W_ks_, squeezed)
        print(Ks.shape)
        Vs = torch.matmul(W_vs_, squeezed)
        print(Vs.shape)

        Qs_Ks = torch.bmm(Qs.transpose(1,2), Ks)/math.sqrt(Ks.shape[1])

        print(Qs_Ks.shape)

        mask = torch.zeros(self.max_seq_len, self.max_seq_len)
        #softmax_op = nn.functional.softmax(Qs_Ks, dim = 1)
        #mask[:, -num_padded_tokens:] = -float('inf')
        if num_padded_tokens != 0:
            mask[-num_padded_tokens:,:] = -float('inf')
        print('MASK', mask)
        print("MASK SHAPE: ",mask.shape)
        softmax_op = nn.functional.softmax(Qs_Ks + mask, dim = 1) #With mask added
        print('SOFTMAX OUTPUT: ',softmax_op)

        print(softmax_op.shape)

        output = torch.bmm(softmax_op, Vs.transpose(1,2))

        print(output.shape)

        head_outputs = [head_op for head_op in output]

        final_output = torch.cat(tuple(head_outputs), dim = 1)

        print(final_output.shape)

        #YOU CONCATENATED THE OUTPUS OF EACH HEAD, BUT DIDN'T APPLY A LINEAR LAYER AFTERWORDS, directly went to Add Norm Layer
        final_output = self.linear(final_output)
        #return final_output
        return final_output + token_embeddings #Add Layer

        #Qs = W_qs @ token_embeddings
        
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.gamma = torch.ones(d_model)#Should set requires_grad = True
        self.beta = torch.zeros(d_model)#Should set requires_grad = True
        self.eps = eps

    def forward(self, token_embeddings):#max_seq_len * d_model
        token_embeddings_mean = torch.mean(token_embeddings, dim=1, keepdim=True)
        token_embeddings_var = ((token_embeddings - token_embeddings_mean)**2)/token_embeddings.shape[1]
        token_embeddings_normalized = (token_embeddings - token_embeddings_mean)/torch.sqrt(token_embeddings_var + self.eps)
        scaled_shifted_token_embeddings = self.gamma * token_embeddings_normalized + self.beta
        return scaled_shifted_token_embeddings

class FFNN(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.proj_space_dim = d_model*2
        self.lin1 = nn.Linear(d_model, self.proj_space_dim)
        self.lin2 = nn.Linear(self.proj_space_dim, d_model)

    def forward(self,token_embeddings):
        transformed1 = self.lin1(token_embeddings)
        nonlin_transformed = torch.tanh(transformed1)
        transformed2 = self.lin2(nonlin_transformed)
        #return transformed2 
        return transformed2 + token_embeddings #Add Layer
        

class Encoder(nn.Module):#nn.Module is needed to have the powers of nn i.e. BACKPROPAGATION, Computation graph, etc
    def __init__(self, vocab_size = VOCAB_SIZE, max_seq_len = MAX_SEQ_LEN, embedding_dim = EMBEDDING_DIM):#constructor
        super().__init__() #calling constructor of nn.Module to have all it's functionalities available to me
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_IDX) 
        #self.embedding[14] = torch.zeroes()
        with torch.no_grad():
            self.embedding.weight[PAD_IDX] = torch.zeros(embedding_dim)
        self.attention = Attention(max_seq_len, embedding_dim)
        self.mul_attention = Multi_Head_Attention()
        self.layer_norm = LayerNorm(embedding_dim)
        self.ffnn = FFNN(embedding_dim)
        self.max_seq_len = max_seq_len

    def forward(self, tokens, padded_tokens=0):#I will need just token_ids of words as input
        #ALSO NEED TO KEEP TRACK NUMBER OF TOKENS THAT WERE PADDED - THEY WILL HELP ME TO DEFINE THE MASKING MATRIX
        print("TOKENS TOKENS: ",tokens)
        #padded_tokens = self.max_seq_len - len(tokens)
        print("PADDED TOKENS: ",padded_tokens)
        #while len(tokens) < self.max_seq_len:
        #    tokens.append(PAD_IDX)
        token_embeddings = self.embedding.weight[tokens]
        #UPDATE token_embeddings with positional encodings HERE
        
        #contextually_aware_representations = self.attention(token_embeddings)
        contextually_aware_representations = self.mul_attention(token_embeddings = token_embeddings, num_padded_tokens = padded_tokens)
        print("CONTEXTUALLY AWARE REPR: ", contextually_aware_representations)
        if contextually_aware_representations != None:
            print("shape: ", contextually_aware_representations.shape)
        layer_normed_car = self.layer_norm(contextually_aware_representations)
        #return contextually_aware_representations
        print("layer normed car shape: ",layer_normed_car.shape)
        car_after_ffnn = self.ffnn(layer_normed_car)
        layer_normed_car_after_ffnn = self.layer_norm(car_after_ffnn)
        
        #return layer_normed_car
        print("layer normed car after ffnn shape: ",layer_normed_car_after_ffnn.shape)
        return layer_normed_car_after_ffnn
        

In [149]:
sample_ip = "10th November 2000"
#sample_op = "2000-11-10"
def get_ip_token_indices(str):
    tokens = []
    print(str)
    str = str.lower()
    print(str)
    for ch in str:
        tokens.append(ctoi[ch])
    padded_tokens = MAX_SEQ_LEN - len(tokens)
    while len(tokens) < MAX_SEQ_LEN:
        tokens.append(ctoi["<pad>"])
    return tokens, padded_tokens
sample_ip_tokens, num_padded_tokens = get_ip_token_indices(sample_ip)
print(sample_ip_tokens)
print(num_padded_tokens)
encoder = Encoder()
encoder(sample_ip_tokens, num_padded_tokens)

10th November 2000
10th november 2000
[4, 3, 34, 24, 0, 29, 30, 36, 21, 28, 18, 21, 32, 0, 5, 3, 3, 3, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
14
REDUCED DIMENSION:  64
TOKENS TOKENS:  [4, 3, 34, 24, 0, 29, 30, 36, 21, 28, 18, 21, 32, 0, 5, 3, 3, 3, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
PADDED TOKENS:  14
EYLLO
torch.Size([128, 64])
torch.Size([128, 64])
torch.Size([128, 64])
EYLLOSLKDJFLS
torch.Size([2, 64, 64])
torch.Size([2, 64, 64])
torch.Size([2, 64, 64])
torch.Size([32, 128])
torch.Size([32, 64])
W_qs_ shape:  torch.Size([2, 64, 64])
token_embeddings_proj shape:  torch.Size([32, 64])
token_embeddings_proj transpose shape:  torch.Size([64, 32])
token_embeddings_proj.unsqueeze:  torch.Size([1, 64, 32])
torch.Size([2, 64, 32])
torch.Size([2, 64, 32])
torch.Size([2, 64, 32])
torch.Size([2, 32, 32])
MASK tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [-inf, -

tensor([[ 11.3137, -11.3137, -11.3137,  ..., -11.3137, -11.3137, -11.3137],
        [-11.3137, -11.3137,  11.3137,  ...,  11.3137, -11.3137,  11.3137],
        [-11.3137, -11.3137,  11.3137,  ...,  11.3137, -11.3137,  11.3137],
        ...,
        [-11.3137, -11.3137, -11.3137,  ...,  11.3137,  11.3137, -11.3137],
        [-11.3137, -11.3137, -11.3137,  ...,  11.3137,  11.3137, -11.3137],
        [-11.3137, -11.3137, -11.3137,  ...,  11.3137,  11.3137, -11.3137]],
       grad_fn=<AddBackward0>)

In [29]:
temp = torch.randint(0,10,(2,4,3))
print(temp)
print(temp.shape)
temp2 = torch.randint(0,10,(1,4,3))
print(temp2)
print(temp2.shape)

print("TEMP2, can transpose be recovered: ",temp2.view(-1,3,4))

print("TEMP2's transpose recover: ",temp2.transpose(1,2),temp2.transpose(1,2).shape)

temp3 = temp * temp2
print(temp3)
print(temp3.shape)

temp4 = torch.randint(0,10,(1,3,4))
print(temp4)
print(temp4.shape)
temp5 = temp @ temp4
print(temp5)
print(temp5.shape)

print(torch.bmm(torch.cat((temp2.transpose(1,2), temp2.transpose(1,2))), temp))

tensor([[[7, 8, 4],
         [5, 5, 1],
         [6, 3, 6],
         [0, 2, 1]],

        [[7, 9, 9],
         [6, 9, 2],
         [6, 5, 8],
         [2, 2, 4]]])
torch.Size([2, 4, 3])
tensor([[[5, 5, 5],
         [8, 7, 8],
         [9, 0, 8],
         [1, 2, 5]]])
torch.Size([1, 4, 3])
TEMP2, can transpose be recovered:  tensor([[[5, 5, 5, 8],
         [7, 8, 9, 0],
         [8, 1, 2, 5]]])
TEMP2's transpose recover:  tensor([[[5, 8, 9, 1],
         [5, 7, 0, 2],
         [5, 8, 8, 5]]]) torch.Size([1, 3, 4])
tensor([[[35, 40, 20],
         [40, 35,  8],
         [54,  0, 48],
         [ 0,  4,  5]],

        [[35, 45, 45],
         [48, 63, 16],
         [54,  0, 64],
         [ 2,  4, 20]]])
torch.Size([2, 4, 3])
tensor([[[5, 9, 9, 8],
         [4, 6, 8, 1],
         [7, 3, 8, 4]]])
torch.Size([1, 3, 4])
tensor([[[ 95, 123, 159,  80],
         [ 52,  78,  93,  49],
         [ 84,  90, 126,  75],
         [ 15,  15,  24,   6]],

        [[134, 144, 207, 101],
         [ 80, 114, 14

In [44]:
temp = torch.randn(2,4,4)
print(temp)
softmax_res = nn.functional.softmax(temp, dim=1)
print(softmax_res)
print(softmax_res.sum(dim=1))

tensor([[[ 0.0846, -0.1009,  0.8951,  0.6654],
         [ 0.3064,  0.4217, -0.0679,  1.5517],
         [-0.2929,  1.3985,  0.5326, -0.3164],
         [-1.4899, -0.4913, -0.4774, -0.9882]],

        [[ 0.4288, -0.3971, -2.5808,  0.4473],
         [ 1.7646,  1.2280, -1.6735, -0.0080],
         [-0.4540,  0.1563,  0.6804, -0.2936],
         [-1.2752, -0.4248,  0.4525, -0.3085]]])
tensor([[[0.3184, 0.1275, 0.4290, 0.2505],
         [0.3974, 0.2150, 0.1638, 0.6077],
         [0.2183, 0.5711, 0.2985, 0.0938],
         [0.0659, 0.0863, 0.1087, 0.0479]],

        [[0.1852, 0.1138, 0.0199, 0.3875],
         [0.7044, 0.5778, 0.0492, 0.2458],
         [0.0766, 0.1978, 0.5183, 0.1847],
         [0.0337, 0.1106, 0.4126, 0.1820]]])
tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000]])


In [64]:
temp = torch.randint(0,5,(2,2,2))
print(temp)
print(temp.view(1,2,-1))
print(temp.squeeze(0))
print(temp.squeeze(0).shape)
print(torch.cat((temp[0], temp[1]), dim=1))
print(torch.cat(tuple([data for data in temp]), dim=1))

tensor([[[0, 1],
         [1, 0]],

        [[4, 1],
         [1, 4]]])
tensor([[[0, 1, 1, 0],
         [4, 1, 1, 4]]])
tensor([[[0, 1],
         [1, 0]],

        [[4, 1],
         [1, 4]]])
torch.Size([2, 2, 2])
tensor([[0, 1, 4, 1],
        [1, 0, 1, 4]])
tensor([[0, 1, 4, 1],
        [1, 0, 1, 4]])


In [9]:
import torch

temp = torch.randint(0,5,(2,4), dtype=torch.float32)
print(temp)
print(temp.shape)
temp2 = temp.mean(dim=1)
print(temp2)
print(temp2.shape)
temp3 = temp.mean(dim=1, keepdim=True)
print(temp3)
print(temp3.shape)
#print(torch.mean(temp, dim=1, keepdim=True)) (just checking syntax variation )
#temp4 = temp - temp2#The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1
#print(temp4)
#print(temp4.shape)
temp5 = temp - temp3
print(temp5)
print(temp5.shape)

tensor([[1., 2., 0., 3.],
        [1., 4., 3., 4.]])
torch.Size([2, 4])
tensor([1.5000, 3.0000])
torch.Size([2])
tensor([[1.5000],
        [3.0000]])
torch.Size([2, 1])
tensor([[-0.5000,  0.5000, -1.5000,  1.5000],
        [-2.0000,  1.0000,  0.0000,  1.0000]])
torch.Size([2, 4])


In [59]:
mask = torch.zeros(8,8)
padded_tokens = 3
mask[:, -padded_tokens] = -float('inf')
print(mask)

tensor([[0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.],
        [0., 0., 0., 0., 0., -inf, 0., 0.]])


Decoder architecture details:<br>
MAX_SEQ_LEN_DEC = 11 specifically => YYYY-MM-DD and \<eos> token<br>
d_model = 128 (Same as the encoder)<br>
Will create a different vocabulary for decoder as only digits, '-', \<sos> and \<eos> (I am posing an easier problem for the decoder to learn from - rather than using our earlier vocabulary containing alphabets, other special symbols, )

In [69]:
temp = torch.zeros(11,11)
temp2 = temp.masked_fill(torch.triu(torch.ones_like(temp), diagonal=1) == 1, float('-inf'))

print(temp2)


tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [101]:
MAX_SEQ_LEN_DEC = 11
EMBEDDING_DIM = 128 #d_model
ATTENTION_HEADS = 2
vocab = "0123456789-"
vocab_list_dec = list(vocab)
vocab_list_dec.append('<sos')
vocab_list_dec.append('<eos>')
vocab_list_dec.sort()
print(vocab_list_dec)
VOCAB_SIZE_DEC = len(vocab_list_dec)
ctoi_dec = {ch:i for i,ch in enumerate(vocab_list_dec)}
print(ctoi_dec)
itoc_dec = {i:ch for i,ch in enumerate(vocab_list_dec)}
print(itoc_dec)

['-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '<eos>', '<sos']
{'-': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '<eos>': 11, '<sos': 12}
{0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9', 11: '<eos>', 12: '<sos'}


Cross Masked Multi Head Attention details : <br>
It will receive op_embedding (from masked multi head attention block) - token_embeddings<br>
It will receive car_enc (contextually aware representations obtained from encoder block) - car_enc <br>
token_embeddings = 11 * 128 <br>
car_enc = 32 * 128 <br>
W_q = 128 * 64 <br>
W_v = 128 * 64 <br>
W_k = 128 * 64 <br>
Since, I want two attention heads <br>
W_qs = (2, 64, 64) - these are two parameter matrices for two attention heads <br>
W_ks = (2, 64, 64) - these are two parameter matrices for two attention heads <br>
W_vs = (2, 64, 64) - these are two parameter matrices for two attention heads <br>

I am going to project my token_embeddings and car_enc from 128 dimensional space to 64 dim space (so that they can be fed to the attention heads) via Linear projection
token_embeddings = 11 * 64<br>
car_enc = 32 * 64 <br>

Qs = W_qs @ token_embeddings.T (2, 64, 11) <br>
Ks = W_ks @ car_enc.T (2, 64, 32) <br>
Vs = W_vs @ car_enc.T (2, 64, 32) <br>

Implementing Attention Equation: <br>
Qs^T Ks / sqrt(d_k) = (2, 11, 32) <br>
softmax across rows in above matrix of 11 * 32 <br>
softmax_op @ Vs.T = (2, 11, 64) <br>
concatenate: (11, 128)  (Your final embeddings) <br>

In [150]:
'''
class Attention(nn.Module):
    def __init__(self, max_seq_len = MAX_SEQ_LEN, ip_embedding_dim = EMBEDDING_DIM, proj_embedding_dim = EMBEDDING_DIM):
        super().__init__()
        #self.W_q = torch.randn(max_seq_len, embedding_dim)
        #self.W_k = torch.randn(max_seq_len, embedding_dim)
        #self.W_v = torch.randn(max_seq_len, embedding_dim)
        self.W_q = torch.randn(ip_embedding_dim, proj_embedding_dim)
        self.W_k = torch.randn(ip_embedding_dim, proj_embedding_dim)
        self.W_v = torch.randn(ip_embedding_dim, proj_embedding_dim)
'''


class Masked_Multi_Head_Attention(nn.Module):
    def __init__(self, heads = ATTENTION_HEADS, ip_embedding_dim = EMBEDDING_DIM, max_seq_len = MAX_SEQ_LEN_DEC):
        super().__init__()
        self.heads = heads
        print("REDUCED DIMENSION: ", int(ip_embedding_dim/self.heads))
        self.attention_heads = [Attention(ip_embedding_dim = int(ip_embedding_dim/self.heads), proj_embedding_dim = int(ip_embedding_dim/self.heads)) for i in range(heads)]
        self.W_qs = [head.W_q for head in self.attention_heads]#learnable
        self.W_ks = [head.W_k for head in self.attention_heads]#learnable
        self.W_vs = [head.W_v for head in self.attention_heads]#learnable
        self.ip_embedding_dim = ip_embedding_dim
        self.proj_embedding_dim = int(ip_embedding_dim/heads)
        self.token_projection = nn.Linear(ip_embedding_dim, self.proj_embedding_dim)
        self.max_seq_len = max_seq_len
        self.linear = nn.Linear(ip_embedding_dim, ip_embedding_dim)
        #self.mask = torch.zeros(max_seq_len * max_seq_len)#NOT LEARNABLE
        #self.mask[:, -num_padded_tokens] = -float('inf')

    def forward(self, token_embeddings):
        #I was thinking of computing contextually aware repr from each of the attention heads one-by-one and then concatenating them
        #This will work and it's implementation is also pretty striaghtforward, but this defeats the purpose of parallelization
        #which can be achieved across the attention heads
        W_qs_ = torch.cat(tuple(self.W_qs), dim = 0)
        W_ks_ = torch.cat(tuple(self.W_ks), dim = 0)
        W_vs_ = torch.cat(tuple(self.W_vs), dim = 0)
        print("EYLLO")
        print(W_qs_.shape)
        print(W_ks_.shape)
        print(W_vs_.shape)

        print("EYLLOSLKDJFLS")

        W_qs_ = W_qs_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim)
        W_ks_ = W_ks_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim)
        W_vs_ = W_vs_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim)

        print(W_qs_.shape)
        print(W_ks_.shape)
        print(W_vs_.shape)

        print(token_embeddings.shape)

        token_embeddings_proj = self.token_projection(token_embeddings)

        print(token_embeddings_proj.shape)

        print("W_qs_ shape: ",W_qs_.shape)
        print("token_embeddings_proj shape: ",token_embeddings_proj.shape)
        print("token_embeddings_proj transpose shape: ",token_embeddings_proj.T.shape)

        #Qs = W_qs_ @ token_embeddings_proj.T
        #print(Qs.shape)
        #Ks = W_ks_ @ token_embeddings_proj.T
        #print(Ks.shape)
        squeezed = token_embeddings_proj.T.unsqueeze(0)
        print("token_embeddings_proj.unsqueeze: ", squeezed.shape )
        Qs = torch.matmul(W_qs_, squeezed)
        print(Qs.shape)
        Ks = torch.matmul(W_ks_, squeezed)
        print(Ks.shape)
        Vs = torch.matmul(W_vs_, squeezed)
        print(Vs.shape)

        Qs_Ks = torch.bmm(Qs.transpose(1,2), Ks)/math.sqrt(Ks.shape[1])

        print(Qs_Ks.shape)

        mask = torch.zeros(self.max_seq_len, self.max_seq_len)
        #softmax_op = nn.functional.softmax(Qs_Ks, dim = 1)
        #mask[:, -num_padded_tokens:] = -float('inf')
        
        #mask[-num_padded_tokens:,:] = -float('inf') Padding mask for encoder
        mask = mask.masked_fill(torch.triu(torch.ones_like(mask), diagonal=1) == 1, float('-inf'))
        
        print('MASK', mask)
        print("MASK SHAPE: ",mask.shape)
        softmax_op = nn.functional.softmax(Qs_Ks + mask, dim = 1) #With mask added
        print('SOFTMAX OUTPUT: ',softmax_op)

        print(softmax_op.shape)

        output = torch.bmm(softmax_op, Vs.transpose(1,2))

        print(output.shape)

        head_outputs = [head_op for head_op in output]

        final_output = torch.cat(tuple(head_outputs), dim = 1)

        print(final_output.shape)

        #YOU CONCATENATED THE OUTPUS OF EACH HEAD, BUT DIDN'T APPLY A LINEAR LAYER AFTERWORDS, directly went to Add Norm Layer
        final_output = self.linear(final_output)
        #return final_output
        return final_output + token_embeddings #Add Layer

        #Qs = W_qs @ token_embeddings

class Cross_Masked_MultiHead_Attention(nn.Module):
    def __init__(self, heads = ATTENTION_HEADS, ip_embedding_dim =  EMBEDDING_DIM, max_seq_len = MAX_SEQ_LEN_DEC):
        super().__init__()
        self.heads = heads
        self.attention_heads = [Attention(ip_embedding_dim = int(ip_embedding_dim/heads), proj_embedding_dim = int(ip_embedding_dim/heads)) for i in range(heads)]
        self.W_qs = [head.W_q for head in self.attention_heads]
        self.W_ks = [head.W_k for head in self.attention_heads]#learnable
        self.W_vs = [head.W_v for head in self.attention_heads]#learnable
        self.ip_embedding_dim = ip_embedding_dim
        self.proj_embedding_dim = int(ip_embedding_dim/heads)
        self.token_projection = nn.Linear(ip_embedding_dim, self.proj_embedding_dim)
        self.max_seq_len = max_seq_len
        self.linear = nn.Linear(ip_embedding_dim, ip_embedding_dim)
        #self.mask = torch.zeros(max_seq_len * max_seq_len)#NOT LEARNABLE
        #self.mask[:, -num_padded_tokens] = -float('inf')        
        

    def forward(self, car_enc, token_embeddings):#contextually aware representations obtained from encoder and op token embeddings
        if token_embeddings is None:#Inference wala code yaha pe ayega
            return torch.tensor([0] * self.max_seq_len)

        print("HUHAHAHA",len(self.W_qs), self.W_qs[0].shape)

        W_qs_ = torch.cat(tuple(self.W_qs), dim = 0) #(ip_embedding_dim, proj_embedding_dim)
        W_ks_ = torch.cat(tuple(self.W_ks), dim = 0) #(ip_embedding_dim, proj_embedding_dim)
        W_vs_ = torch.cat(tuple(self.W_vs), dim = 0) #(ip_embedding_dim, proj_embedding_dim)
        print("EYLLO")
        print(W_qs_.shape)
        print(W_ks_.shape)
        print(W_vs_.shape)

        print("EYLLOSLKDJFLS")

        W_qs_ = W_qs_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim) #(#heads, proj_embedding_dim, proj_embedding_dim)
        W_ks_ = W_ks_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim) #(#heads, proj_embedding_dim, proj_embedding_dim)
        W_vs_ = W_vs_.view(-1, self.proj_embedding_dim, self.proj_embedding_dim) #(#heads, proj_embedding_dim, proj_embedding_dim)

        print(W_qs_.shape)
        print(W_ks_.shape)
        print(W_vs_.shape)

        print(token_embeddings.shape)

        token_embeddings_proj = self.token_projection(token_embeddings)

        print(token_embeddings_proj.shape)

        print(car_enc.shape)

        car_enc_proj = self.token_projection(car_enc)

        print(car_enc_proj.shape)

        unsqueezed_token_embeddings_proj = token_embeddings_proj.T.unsqueeze(0)
        print("token_embeddings_proj.unsqueeze: ", unsqueezed_token_embeddings_proj.shape )
        Qs = torch.matmul(W_qs_, unsqueezed_token_embeddings_proj)
        print(Qs.shape)

        unsqueezed_car_enc_proj = car_enc_proj.T.unsqueeze(0)

        print("car_enc_proj.unsqueeze: ", unsqueezed_car_enc_proj.shape)
        print("W_ks_: ",W_ks_)
        print("unsqueezed_car_enc_proj: ",unsqueezed_car_enc_proj)
        Ks = torch.matmul(W_ks_, unsqueezed_car_enc_proj)
        print(Ks.shape)

        print("Qs: ",Qs)
        print("Ks: ",Ks)

        Vs = torch.matmul(W_vs_, unsqueezed_car_enc_proj)
        print(Vs.shape)

        #Attention equation implementation starts
        Qs_Ks = torch.bmm(Qs.transpose(1,2), Ks)/math.sqrt(Ks.shape[1])

        print("QS_KS: ",Qs_Ks)

        print(Qs_Ks.shape)

        #There is no mask needed at cross multi head attention

        '''
        mask = torch.zeros(token_embeddings.shape[0], car_enc.shape[0])

        print(mask.shape)

        mask = mask.masked_fill(torch.triu(torch.ones_like(mask), diagonal=1) == 1, float('-inf'))

        print(mask)
        '''

        softmax_op = nn.functional.softmax(Qs_Ks, dim = 1)
        print(softmax_op.shape)

        output = torch.bmm(softmax_op, Vs.transpose(1,2))

        print(output.shape)

        head_outputs = [head_op for head_op in output]

        final_output = torch.cat(tuple(head_outputs), dim = 1)

        print(final_output.shape)
        #INSERT A LINEAR LAYER(with dimensions = (ip_embedding, ip_embedding_dim)) HERE ON final_output, so that the concatenated learnt representations are nicely mixed

        final_output = self.linear(final_output)

        #print(final_output.shape)
    
        return final_output + token_embeddings

class Decoder(nn.Module):
    def __init__(self, max_seq_len = MAX_SEQ_LEN_DEC, vocab_size = VOCAB_SIZE_DEC, d_model = EMBEDDING_DIM):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.embedding_layer = nn.Embedding(vocab_size, d_model)
        self.masked_multihead = Masked_Multi_Head_Attention()
        self.layer_norm = LayerNorm(d_model)
        self.cross_masked_multihead = Cross_Masked_MultiHead_Attention()
        self.ffnn = FFNN(d_model)
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, car_enc, tokens=None):
        #token_embeddings = self.embedding_layer(tokens)
        #token_embeddings = self.embedding_layer(torch.tensor(tokens))
        if tokens is None:#Inference vala code ayega yaha pe
            return torch.tensor([0] * self.max_seq_len)
        token_embeddings = self.embedding_layer(torch.tensor(tokens))
        print(token_embeddings)
        print(token_embeddings.shape)
        car_1 = self.masked_multihead(token_embeddings)
        print(car_1)
        print(car_1.shape)
        car_1_norm = self.layer_norm(car_1)
        print(car_1_norm)
        print(car_1_norm.shape)

        cross_attended_car = self.cross_masked_multihead(car_enc, car_1_norm)
        print(cross_attended_car.shape)

        cross_attended_car_norm = self.layer_norm(cross_attended_car)
        print(cross_attended_car_norm.shape)

        car_after_ffnn = self.ffnn(cross_attended_car_norm)
        print(car_after_ffnn.shape)

        car_after_ffnn_layer_norm = self.layer_norm(car_after_ffnn)
        print(car_after_ffnn_layer_norm.shape)

        logits = self.linear(car_after_ffnn_layer_norm)
        print("LOGITS: ",logits.shape)

        softmax_op = nn.functional.softmax(logits, dim=1)
        print(softmax_op.shape)
        
        return logits
        
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, ip_tokens, op_tokens=None, num_padded_tokens_in_ip=0):#op_tokens will be None during inference and it will contain op_tokens during training (for teacher forcing during initial stages)
        car_enc = self.encoder(tokens = ip_tokens, padded_tokens = num_padded_tokens_in_ip)
        print("car_enc type: ",type(car_enc))
        print("car_enc: ",car_enc)
        print("car_enc shape: ",car_enc.shape)
        final_op_tokens = self.decoder(car_enc, op_tokens)#should contain token indices, particularly 11 token indices
        return final_op_tokens
        
        
        

In [156]:
decoder = Decoder()
date_example = "1947-08-15"
token_ids = []
for ch in date_example:
    token_ids.append(ctoi[ch])
token_ids.append(ctoi_dec["<eos>"])
dummy_car_enc_token_ids = torch.randn(32,128)
#dummy_car_enc_token_ids = [0] * 32 (wrong since, we need to feed 32 car_enc embeddings)
print(dummy_car_enc_token_ids.shape)
decoder(dummy_car_enc_token_ids, token_ids)

REDUCED DIMENSION:  64
torch.Size([32, 128])
tensor([[-0.0069,  1.7548, -1.1532,  ...,  1.2560,  0.5332, -2.1933],
        [-1.5602, -0.7389, -0.3998,  ..., -0.2399,  0.2198,  1.0772],
        [ 0.5438, -0.3836, -0.6168,  ...,  3.2215,  0.8302, -1.3361],
        ...,
        [-0.0069,  1.7548, -1.1532,  ...,  1.2560,  0.5332, -2.1933],
        [-0.2212, -0.5800, -0.2191,  ..., -0.0268,  0.7745,  0.8087],
        [-0.7779, -0.6317,  0.9720,  ...,  0.3528,  1.0134, -0.5293]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([11, 128])
EYLLO
torch.Size([128, 64])
torch.Size([128, 64])
torch.Size([128, 64])
EYLLOSLKDJFLS
torch.Size([2, 64, 64])
torch.Size([2, 64, 64])
torch.Size([2, 64, 64])
torch.Size([11, 128])
torch.Size([11, 64])
W_qs_ shape:  torch.Size([2, 64, 64])
token_embeddings_proj shape:  torch.Size([11, 64])
token_embeddings_proj transpose shape:  torch.Size([64, 11])
token_embeddings_proj.unsqueeze:  torch.Size([1, 64, 11])
torch.Size([2, 64, 11])
torch.Size([2, 64, 11])
torch

tensor([[  0.8159,   0.9526,   4.0626,  -1.0285,  -0.1347, -10.2289,  -0.4143,
          -1.1155,  11.8756,  -1.2605,   4.3895,  -4.9434,  -1.0471],
        [  3.8913,  -4.6237,   2.3252,   3.2112,   0.0979, -10.1650,   1.0539,
           2.2344,  -1.3694,  -3.1331,   1.2873,   1.1184,  -3.4961],
        [ -6.9505,   8.9966,   1.0496,   9.6139,   4.7723,   0.4509, -11.6586,
           4.5179, -12.4108,  -6.4686,   8.3179,   4.9169,  -1.5999],
        [  7.5333,  12.1417,   5.9286,  -0.7985,  -0.7742,  -2.7438,   0.0250,
           5.4299,  -1.6144,  -7.3389,   3.8814, -11.3878,  -0.6786],
        [ -8.2901,   3.3453,  -2.4744,  -9.6144,  -3.8608,  -2.4068,  -7.4242,
           5.7768,  -9.1445,  -0.2286,   2.5977, -15.7328,  -1.4880],
        [  3.9572,  -2.3089,   3.8112, -15.9238,  -9.5287, -13.7192,  -7.4584,
           1.7347,   0.7506,   4.7978,  -3.3110,   3.7245,  -1.4338],
        [ -3.3364,  11.3614,  -0.7551,   5.4343,   4.2162,  -4.3309,   1.7045,
          13.6422,  -2.9986

In [146]:
sample_ip = "10th November 2000"
sample_op = "2000-11-10"
def get_ip_token_indices(str):
    tokens = []
    print(str)
    str = str.lower()
    print(str)
    for ch in str:
        tokens.append(ctoi[ch])
    padded_tokens = MAX_SEQ_LEN - len(tokens)
    while len(tokens) < MAX_SEQ_LEN:
        tokens.append(ctoi["<pad>"])
    return tokens, padded_tokens

def get_op_token_indices(str):
    tokens = []
    for ch in str:
        tokens.append(ctoi_dec[ch])
    tokens.append(ctoi_dec["<eos>"])
    return tokens
sample_ip_tokens, padded_tokens = get_ip_token_indices(sample_ip)
sample_op_tokens = get_op_token_indices(sample_op)
print(sample_ip_tokens)
print(padded_tokens)
print(sample_op_tokens)

10th November 2000
10th november 2000
[4, 3, 34, 24, 0, 29, 30, 36, 21, 28, 18, 21, 32, 0, 5, 3, 3, 3, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
14
[3, 1, 1, 1, 0, 2, 2, 0, 2, 1, 11]


In [158]:
transformer = Transformer()
probabilities = transformer(ip_tokens = sample_ip_tokens, op_tokens = sample_op_tokens, num_padded_tokens_in_ip = padded_tokens)

REDUCED DIMENSION:  64
REDUCED DIMENSION:  64
TOKENS TOKENS:  [4, 3, 34, 24, 0, 29, 30, 36, 21, 28, 18, 21, 32, 0, 5, 3, 3, 3, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
PADDED TOKENS:  14
EYLLO
torch.Size([128, 64])
torch.Size([128, 64])
torch.Size([128, 64])
EYLLOSLKDJFLS
torch.Size([2, 64, 64])
torch.Size([2, 64, 64])
torch.Size([2, 64, 64])
torch.Size([32, 128])
torch.Size([32, 64])
W_qs_ shape:  torch.Size([2, 64, 64])
token_embeddings_proj shape:  torch.Size([32, 64])
token_embeddings_proj transpose shape:  torch.Size([64, 32])
token_embeddings_proj.unsqueeze:  torch.Size([1, 64, 32])
torch.Size([2, 64, 32])
torch.Size([2, 64, 32])
torch.Size([2, 64, 32])
torch.Size([2, 32, 32])
MASK tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]

In [138]:
temp = [1,2,3]
def func(x):
    x.append(4)
func(temp)
print(temp)
#If you ran decoder earlier (decoder = Decoder()) snippet earlier, then those tokens were modified with padding to matcht the max_seq_len
#Secondly, when you had a sequence length of exactly equal to MAX_SEQ_LEN (then, there was a small bug while filling the mask : mas[-num_padded_token:0, :])
#so, if you had 0 padded tokens, this snippet was initializing all of the rows to inf in mask, I fixed that, so the first bug, help me identify second bug and in end both were resolved

#NOW, we need to implement a linear layer that maps 128 sized embedding to a VOCAB_SIZE_LEN_DEC space and then apply softmax over that final step to obtain probabilities over the vocab
# We need to do this for all the 11 output embeddings obtained from the decoder.

[1, 2, 3, 4]


In [159]:
max_indices = torch.argmax(probabilities, dim=1)

# Convert the result to a Python list if needed
max_indices_list = max_indices.tolist()
max_indices_list

[1, 3, 5, 10, 5, 7, 9, 10, 2, 4, 12]

In [160]:
decoded_tokens = [itoc_dec[idx] for idx in max_indices_list]
decoded_tokens

['0', '2', '4', '9', '4', '6', '8', '9', '1', '3', '<sos']

In [162]:
print(''.join(decoded_tokens))
#Expected output: 2000-11-10<eos>
#Hopefully, after proper training with training data, this will improve :)

0249468913<sos
